<a href="https://colab.research.google.com/github/aj1365/DeepForestPaper/blob/main/DeepForest_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
######################################### Connecting to Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/DForest

In [ ]:
############################## Importing the Deep Forest GitHub Repository

!git clone https://github.com/kingfengji/gcForest.git

In [ ]:
############################### We can store it in our Google Drive as well (you need to change the library location)

import sys
sys.path.insert(0,'/content/drive/MyDrive/DForest/gcForest/lib')

In [ ]:
from gcforest.gcforest import GCForest

In [ ]:
import argparse
import numpy as np
import sys
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")


In [ ]:
from sklearn.model_selection import train_test_split

Data = np.loadtxt('DataAvalon2.csv', delimiter=',')

X_train, X_test, y_train, y_test = train_test_split(Data[:,0:27],
                                                    Data[:,28],
                                                    stratify=Data[:,28],
                                                    test_size=0.50,
                                                    random_state=0)

In [ ]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", dest="model", type=str, default=None, help="gcfoest Net Model File")
    args = parser.parse_args()
    return args


In [ ]:
########################################## setting the Deep Forest parameters

def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = 0
    ca_config["max_layers"] = 20
    ca_config["early_stopping_rounds"] = 1
    ca_config["n_classes"] = 8
    ca_config["estimators"] = []
    ca_config["estimators"].append(
            {"n_folds": 5, "type": "XGBClassifier", "n_estimators": 200, "max_depth": 5,
             "objective": "multi:softprob", "silent": True, "nthread": -1, "learning_rate": 0.1} )
    config["cascade"] = ca_config
    return config

In [ ]:
########################################## Training and predicting the classes with the Deep Forest

config=get_toy_config()
gc = GCForest(config)
X_train_enc = gc.fit_transform(X_train, y_train-1)
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test-1, y_pred)
print("Test Accuracy of GcForest = {:.2f} %".format(acc * 100))

In [ ]:
############################################# classification metrices

from sklearn import metrics

print(metrics.classification_report(y_pred, y_test-1))

In [ ]:
############################################# classification confusion matrix

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

mat = confusion_matrix(y_test-1, y_pred)
mat